### Data sandbox 

*Le but ici est de trouver une manière simple de merger la data provenant de Enedis et de l'ADEME en passant par les clés de la BAN. Pour faire simple on part sur le scope : *
* maisons
* 

- source enedis : https://data.enedis.fr/explore/dataset/consommation-annuelle-residentielle-par-adresse/api/
- source ademe : 
- source ban : 

In [1]:
import requests, pandas as pd

In [67]:
def load_get_data_pandas(url):
    res = requests.get(url).json().get('results')
    return pd.DataFrame(res)

get_url_enedis_year_rows = lambda annee, rows: f"https://data.enedis.fr/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/records?where=annee%20%3D%20date'{annee}'&limit={rows}"
get_url_ademe_filtered_on_ban = lambda key: f"https://data.ademe.fr/data-fair/api/v1/datasets/dpe-v2-logements-existants/lines?size=1000&format=json&qs=Identifiant__BAN%3A{key}"

In [7]:
url_ademe = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe-v2-logements-existants/lines?size=1000&format=json&qs=Type_b%C3%A2timent%3Amaison"
url_enedis = "https://data.enedis.fr/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/records?limit=20"

In [9]:
# avec ceci on voit que la plupart des adresses enedis sont des logements 'residentiels' avec au moins 10 logements et au plus +de 1000 logements
tmp_url = "https://data.enedis.fr/api/explore/v2.1/catalog/datasets/consommation-annuelle-residentielle-par-adresse/records?select=segment_de_client%2C%20annee%2C%20count\
    (code_iris)%20as%20nb%2C%20min(nombre_de_logements)%20as%20nb_min_logements%2C%20max(nombre_de_logements)%20as%20nb_max_logements&group_by=segment_de_client%2C%20annee&limit=20"

load_get_data_pandas(tmp_url)

,segment_de_client,annee,nb,nb_min_logements,nb_max_logements
0,RESIDENTIEL,2018-01-01T00:00:00+00:00,391099,10,1189
1,RESIDENTIEL,2019-01-01T00:00:00+00:00,399791,10,1124
2,RESIDENTIEL,2020-01-01T00:00:00+00:00,405605,10,1435
3,RESIDENTIEL,2021-01-01T00:00:00+00:00,407053,10,1189
4,RESIDENTIEL,2022-01-01T00:00:00+00:00,394100,10,1190
5,RESIDENTIEL,2023-01-01T00:00:00+00:00,397398,10,1196


In [20]:
load_get_data_pandas(get_url_enedis_year_rows(2018, 20)).head()

,annee,code_iris,nom_iris,numero_de_voie,indice_de_repetition,type_de_voie,libelle_de_voie,code_commune,nom_commune,segment_de_client,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,consommation_annuelle_moyenne_par_site_de_l_adresse_mwh,consommation_annuelle_moyenne_de_la_commune_mwh,adresse,code_epci,code_departement,code_region,tri_des_adresses
0,2018,160150302,Saint-Martin-Saint-Gelais-l'Anguienne,9,None,IMPASSE,DU PONT DE VARS,16015,ANGOULEME,RESIDENTIEL,60,198.887,3.315,3.139,9 IMPASSE DU PONT DE VARS,200071827,16,75,410003
1,2018,160150101,Plateau Nord-Montausier,20,None,RUE,FROIDE,16015,ANGOULEME,RESIDENTIEL,48,71.683,1.493,3.139,20 RUE FROIDE,200071827,16,75,410012
2,2018,160150501,Bel-Air-Grand Font,5,None,PLACE,HENRI CHAMARRE,16015,ANGOULEME,RESIDENTIEL,37,57.947,1.566,3.139,5 PLACE HENRI CHAMARRE,200071827,16,75,410023
3,2018,160150704,Petite Garenne,10,None,PLACE,HENRI FARMAN,16015,ANGOULEME,RESIDENTIEL,19,25.515,1.343,3.139,10 PLACE HENRI FARMAN,200071827,16,75,410025
4,2018,160150501,Bel-Air-Grand Font,3,None,IMPASSE,JACKY HUMBLOT,16015,ANGOULEME,RESIDENTIEL,84,105.189,1.252,3.139,3 IMPASSE JACKY HUMBLOT,200071827,16,75,410032


In [70]:
load_get_data_pandas(url_ademe).head() 

,Conso_chauffage_dépensier_é_finale,Volume_stockage_générateur_ECS_n°1,Conso_ECS_é_finale_énergie_n°3,Conso_é_finale_installation_ECS,Nom__commune_(BAN),Emission_GES_chauffage,Conso_ECS_é_finale_énergie_n°2,Conso_ECS_é_finale_énergie_n°1,Besoin_refroidissement,Conso_chauffage_dépensier_installation_chauffage_n°1,...,Type_générateur_n°2_installation_n°2,Usage_générateur_n°2_installation_n°2,Type_énergie_générateur_n°2_installation_n°2,Invariant_fiscal_logement,Position_logement_dans_immeuble,Typologie_logement,Surface_habitable_immeuble,Surface_tertiaire_immeuble,Facteur_couverture_solaire_installation_chauffage_n°2,Facteur_couverture_solaire_saisi
0,19939.8,0.0,0.0,2406.9,Bouxwiller,3840.0,0.0,2406.9,0,19939.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5921.3,200.0,NaN,2168.0,Morlanne,365.8,NaN,2100.1,"666,4",3433.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,54188.8,900.0,NaN,767.8,Gentilly,14320.1,0.0,767.8,0,54188.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8053.0,0.0,NaN,1702.3,Liffré,1393.7,0.0,1702.3,0,8053.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18182.9,100.0,NaN,2808.0,Saint-Sébastien-de-Morsent,3335.2,0.0,2808.0,0,18182.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# avec ademe, les codes BAN sont deja fournis 
[_ for _ in load_get_data_pandas(url_ademe).columns if ('BAN' in _) or ('adresse' in _.lower())]

['Nom__commune_(BAN)',
 'Coordonnée_cartographique_X_(BAN)',
 'Adresse_(BAN)',
 'N°_voie_(BAN)',
 'Adresse_brute',
 'N°_département_(BAN)',
 'N°_région_(BAN)',
 'Coordonnée_cartographique_Y_(BAN)',
 'Code_postal_(BAN)',
 'Identifiant__BAN',
 'Nom__rue_(BAN)',
 'Code_INSEE_(BAN)',
 'Score_BAN',
 "Complément_d'adresse_bâtiment",
 "Complément_d'adresse_logement"]

In [24]:
enedis_adresses_list = list(zip(load_get_data_pandas(get_url_enedis_year_rows(2018, 20)).adresse.values.tolist(),
                                load_get_data_pandas(get_url_enedis_year_rows(2018, 20)).code_commune.values.tolist(),
                                load_get_data_pandas(get_url_enedis_year_rows(2018, 20)).nom_commune.values.tolist()))
enedis_adresses_list = [f"{a} {b} {c}" for a,b,c in enedis_adresses_list]
enedis_adresses_list

['1 RUE PAUL VERLAINE 21231 DIJON',
 '9 RUE PETITOT 21231 DIJON',
 '8 RUE PIERRE JOSEPH MAGNIN 21231 DIJON',
 '4 RUE PIERRE JOSEPH MAGNIN 21231 DIJON',
 '1 RUE PIERRE PRUDHON 21231 DIJON',
 '28 RUE PIERRE TRAVAUX 21231 DIJON',
 '6 RUE RAMEAU 21231 DIJON',
 '94 AVENUE RAYMOND POINCARE 21231 DIJON',
 '70 AVENUE RAYMOND POINCARE 21231 DIJON',
 '65 AVENUE RAYMOND POINCARE 21231 DIJON',
 '88 B AVENUE RAYMOND POINCARE 21231 DIJON',
 '32 BOULEVARD REMBRANDT 21231 DIJON',
 '18 RUE RETIF DE LA BRETONNE 21231 DIJON',
 '16 RUE RETIF DE LA BRETONNE 21231 DIJON',
 '2 RUE ROBERT DELAUNAY 21231 DIJON',
 '2 ALLEE ROBERT DELAVIGNETTE 21231 DIJON',
 '6 B RUE ROBERT FOLZ 21231 DIJON',
 '24 BOULEVARD ROBERT SCHUMAN 21231 DIJON',
 '4 PLACE SAINT MICHEL 21231 DIJON',
 '4 RUE THEODORE DE BEZE 21231 DIJON']

In [51]:
# pour les adresses obtenues avec enedis on va requeter (post) l'api BAN pour obtenir les nfos sur l'adresse dont le  code BAN
import requests


def get_ban_res(addr):
    ADDOK_URL = 'http://api-adresse.data.gouv.fr/search/'

    params = {
        'q': addr,
        'limit': 1
    }
    response = requests.get(ADDOK_URL, params=params)
    j = response.json()
    if len(j.get('features')) > 0:
        first_result = j.get('features')[0]
        lon, lat = first_result.get('geometry').get('coordinates')
        first_result_all_infos = { **first_result.get('properties'), **{"lon": lon, "lat": lat}, **{'full_adress': addr}}
        return first_result_all_infos
    else:
        return
    
#assert len([get_ban_res(_) for _ in enedis_adresses_list]) == len(enedis_adresses_list)
pd.DataFrame([get_ban_res(_) for _ in enedis_adresses_list])

,label,score,housenumber,id,name,postcode,citycode,x,y,city,context,type,importance,street,lon,lat,full_adress
0,1 Rue Paul Verlaine 21000 Dijon,0.711316,1,21231_6747_00001,1 Rue Paul Verlaine,21000,21231,854384.30,6696686.03,Dijon,"21, Côte-d'Or, Bourgogne-Franche-Comté",housenumber,0.62448,Rue Paul Verlaine,5.045308,47.352511,1 RUE PAUL VERLAINE 21231 DIJON
1,9 Rue Petitot 21000 Dijon,0.675970,9,21231_6910_00009,9 Rue Petitot,21000,21231,854004.18,6694214.87,Dijon,"21, Côte-d'Or, Bourgogne-Franche-Comté",housenumber,0.68567,Rue Petitot,5.039429,47.330361,9 RUE PETITOT 21231 DIJON
2,8 Rue Pierre Joseph Magnin 21000 Dijon,0.743805,8,21231_7031_00008,8 Rue Pierre Joseph Magnin,21000,21231,854575.73,6696636.87,Dijon,"21, Côte-d'Or, Bourgogne-Franche-Comté",housenumber,0.64131,Rue Pierre Joseph Magnin,5.047826,47.352024,8 RUE PIERRE JOSEPH MAGNIN 21231 DIJON
3,4 Rue Pierre Joseph Magnin 21000 Dijon,0.743805,4,21231_7031_00004,4 Rue Pierre Joseph Magnin,21000,21231,854548.13,6696562.49,Dijon,"21, Côte-d'Or, Bourgogne-Franche-Comté",housenumber,0.64131,Rue Pierre Joseph Magnin,5.047435,47.351361,4 RUE PIERRE JOSEPH MAGNIN 21231 DIJON
4,1 Rue Pierre Prudhon 21000 Dijon,0.719746,1,21231_7380_00001,1 Rue Pierre Prudhon,21000,21231,854096.89,6693729.34,Dijon,"21, Côte-d'Or, Bourgogne-Franche-Comté",housenumber,0.65914,Rue Pierre Prudhon,5.040490,47.325970,1 RUE PIERRE PRUDHON 21231 DIJON
5,28 Rue Pierre Travaux 21000 Dijon,0.730517,28,21231_7070_00028,28 Rue Pierre Travaux,21000,21231,855431.00,6692283.63,Dijon,"21, Côte-d'Or, Bourgogne-Franche-Comté",housenumber,0.72319,Rue Pierre Travaux,5.057649,47.312648,28 RUE PIERRE TRAVAUX 21231 DIJON
6,6 Rue Rameau 21000 Dijon,0.665099,6,21231_7480_00006,6 Rue Rameau,21000,21231,854245.20,6693212.36,Dijon,"21, Côte-d'Or, Bourgogne-Franche-Comté",housenumber,0.66391,Rue Rameau,5.042276,47.321283,6 RUE RAMEAU 21231 DIJON
7,94 Avenue Raymond Poincare 21000 Dijon,0.754524,94,21231_7520_00094,94 Avenue Raymond Poincare,21000,21231,856205.81,6694093.14,Dijon,"21, Côte-d'Or, Bourgogne-Franche-Comté",housenumber,0.75922,Avenue Raymond Poincare,5.068529,47.328750,94 AVENUE RAYMOND POINCARE 21231 DIJON
8,70 Avenue Raymond Poincare 21000 Dijon,0.754524,70,21231_7520_00070,70 Avenue Raymond Poincare,21000,21231,855978.91,6694085.53,Dijon,"21, Côte-d'Or, Bourgogne-Franche-Comté",housenumber,0.75922,Avenue Raymond Poincare,5.065523,47.328735,70 AVENUE RAYMOND POINCARE 21231 DIJON
9,65 Avenue Raymond Poincare 21000 Dijon,0.754524,65,21231_7520_00065,65 Avenue Raymond Poincare,21000,21231,856242.15,6694121.87,Dijon,"21, Côte-d'Or, Bourgogne-Franche-Comté",housenumber,0.75922,Avenue Raymond Poincare,5.069020,47.329000,65 AVENUE RAYMOND POINCARE 21231 DIJON


In [63]:
import numpy as np

def get_enedis_with_ban_pandas(requete_url_enedis): 
    enedis_data = load_get_data_pandas(requete_url_enedis)
    enedis_adresses_list = list(zip(enedis_data.adresse.values.tolist(),
                                     enedis_data.code_commune.values.tolist(),
                                     enedis_data.nom_commune.values.tolist()))
    enedis_adresses_list = [f"{a} {b} {c}" for a,b,c in enedis_adresses_list]
    enedis_data['full_adress'] = enedis_adresses_list

    # --------- pas optimisé : 1min pour 20 lignes ---- suppr ---------
    #enedis_data['full_adress'] = enedis_data.apply(lambda r: f"{r['adresse']} {r['code_commune']} {r['nom_commune']}", axis=1) 
    #enedis_adresses_list = enedis_data.full_adress.values.tolist()

    ban_data = pd.DataFrame([get_ban_res(_) for _ in enedis_adresses_list])
    vectorized_upper = np.vectorize(str.upper, cache=True)
    ban_data['label'] = vectorized_upper(ban_data['label'].values)
    return pd.merge(enedis_data, ban_data, how='inner', left_on='full_adress', right_on='full_adress').rename(columns={'id': 'id_BAN'})

enedis_with_ban = get_enedis_with_ban_pandas(get_url_enedis_year_rows(2018, 20))

In [65]:
enedis_with_ban.head(3)

,annee,code_iris,nom_iris,numero_de_voie,indice_de_repetition,type_de_voie,libelle_de_voie,code_commune,nom_commune,segment_de_client,...,city,context,type,importance,street,lon,lat,oldcitycode,oldcity,banId
0,2018,160150302,Saint-Martin-Saint-Gelais-l'Anguienne,9,None,IMPASSE,DU PONT DE VARS,16015,ANGOULEME,RESIDENTIEL,...,Angoulême,"16, Charente, Nouvelle-Aquitaine",housenumber,0.60820,Impasse du Pont de Vars,0.158184,45.642832,NaN,NaN,NaN
1,2018,160150101,Plateau Nord-Montausier,20,None,RUE,FROIDE,16015,ANGOULEME,RESIDENTIEL,...,Angoulême,"16, Charente, Nouvelle-Aquitaine",housenumber,0.64461,Rue Froide,0.153120,45.650905,NaN,NaN,NaN
2,2018,160150501,Bel-Air-Grand Font,5,None,PLACE,HENRI CHAMARRE,16015,ANGOULEME,RESIDENTIEL,...,Angoulême,"16, Charente, Nouvelle-Aquitaine",housenumber,0.53117,Place Henri Chamarre,0.173195,45.653916,NaN,NaN,NaN


In [93]:
get_url_ademe_filter_on_ban = lambda key: f'https://data.ademe.fr/data-fair/api/v1/datasets/dpe-v2-logements-existants/lines?format=json&qs=Identifiant__BAN%3A%20%22{key}%22'


In [94]:
get_url_ademe_filter_on_ban('13214_5989_00129')

'https://data.ademe.fr/data-fair/api/v1/datasets/dpe-v2-logements-existants/lines?format=json&qs=Identifiant__BAN%3A%20%2213214_5989_00129%22'

In [81]:
#requests.get('https://data.ademe.fr/data-fair/api/v1/datasets/dpe-v2-logements-existants/lines?size=2&format=json&qs=Identifiant__BAN%3A%20%2213214598900129%22').json()
requests.get(get_url_ademe_filter_on_ban('13214_5989_00129')).json()

{'total': 183,
 'next': 'https://data.ademe.fr/data-fair/api/v1/datasets/dpe-v2-logements-existants/lines?size=2&format=json&qs=Identifiant__BAN%3A+%2213214_5989_00129%22&after=331%2C337326',
 'results': [{'Conso_chauffage_dépensier_é_finale': 396.6,
   'Volume_stockage_générateur_ECS_n°1': 150,
   'Conso_é_finale_installation_ECS': 1845.3,
   'Nom__commune_(BAN)': 'Marseille',
   'Emission_GES_chauffage': 27,
   'Conso_ECS_é_finale_énergie_n°1': 1124.1,
   'Besoin_refroidissement': '0',
   'Conso_chauffage_dépensier_installation_chauffage_n°1': 647.2,
   'Coût_total_5_usages': 373.2,
   'Configuration_installation_chauffage_n°1': 'Installation de chauffage simple',
   'Conso_é_finale_dépensier_installation_ECS': 2384.8,
   'Configuration_installation_ECS': "Un seul système d'ECS sans solaire",
   'Type_installation_chauffage_n°1': 'installation individuelle',
   'Surface_chauffée_installation_chauffage_n°1': 19.5,
   'Coordonnée_cartographique_X_(BAN)': 895347.85,
   'Nombre_niveau_lo

In [95]:
x = [requests.get(get_url_ademe_filter_on_ban(_)).json().get('results') for _ in enedis_with_ban.id_BAN.values.tolist()]
x

[[{'Conso_chauffage_dépensier_é_finale': 1283.1,
   'Volume_stockage_générateur_ECS_n°1': 150,
   'Conso_é_finale_installation_ECS': 1643.3,
   'Nom__commune_(BAN)': 'Angoulême',
   'Emission_GES_chauffage': 73.1,
   'Conso_ECS_é_finale_énergie_n°1': 1643.3,
   'Besoin_refroidissement': '0',
   'Conso_chauffage_dépensier_installation_chauffage_n°1': 1283.1,
   'Coût_total_5_usages': 709.9,
   'Configuration_installation_chauffage_n°1': 'Installation de chauffage simple',
   'Conso_é_finale_dépensier_installation_ECS': 2099.9,
   'Configuration_installation_ECS': "Un seul système d'ECS sans solaire",
   'Type_installation_chauffage_n°1': 'installation individuelle',
   'Surface_chauffée_installation_chauffage_n°1': 38.3,
   'Coordonnée_cartographique_X_(BAN)': 478683.79,
   'Nombre_niveau_logement': 1,
   'Apports_internes_saison_froid': 0,
   'Type_installation_ECS_(général)': 'individuel',
   'Déperditions_murs': 6.5,
   'Conso_5_usages_par_m²_é_primaire': 170,
   'Coût_refroidissemen

In [101]:
X = []
for _ in x:
    X.extend(_)
len(X)
pd.DataFrame(X)

,Conso_chauffage_dépensier_é_finale,Volume_stockage_générateur_ECS_n°1,Conso_é_finale_installation_ECS,Nom__commune_(BAN),Emission_GES_chauffage,Conso_ECS_é_finale_énergie_n°1,Besoin_refroidissement,Conso_chauffage_dépensier_installation_chauffage_n°1,Coût_total_5_usages,Configuration_installation_chauffage_n°1,...,Type_émetteur_installation_chauffage_n°2,Description_générateur_chauffage_n°1_installation_n°2,Facteur_couverture_solaire_saisi,Conso_chauffage_générateur_n°1_installation_n°2,Conso_chauffage_installation_chauffage_n°2,Usage_générateur_n°1_installation_n°2,Type_énergie_générateur_n°1_installation_n°2,Conso_chauffage_dépensier_générateur_n°1_installation_n°2,Type_générateur_n°1_installation_n°2,Electricité_PV_autoconsommée
0,1283.1,150.0,1643.3,Angoulême,73.1,1643.3,0,1283.1,709.9,Installation de chauffage simple,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1416.6,150.0,1571.8,Angoulême,82.4,1571.8,0,1416.6,558.0,Installation de chauffage simple,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,953.6,150.0,1698.5,Angoulême,54.4,1698.5,0,953.6,525.0,Installation de chauffage simple,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1249.3,NaN,NaN,Angoulême,72.1,1114.5,0,NaN,659.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1124.1,NaN,NaN,Angoulême,67.1,1058.4,0,NaN,626.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,5325.8,0.0,76078.9,Annecy,982.0,1724.4,0,286445.9,806.0,Installation de chauffage simple,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,15967.8,100.0,1.0,Annecy,2954.8,1918.9,0,1.0,1444.2,Installation de chauffage simple,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
105,28706.3,5.1,5066.7,Annecy,5313.9,0.0,0,28706.3,2357.5,Installation de chauffage simple,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,19898.2,150.0,1693.2,Annecy,3750.0,0.0,0,19898.2,1768.0,Installation de chauffage simple,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
def get_enedis_with_ban_with_ademe(requete_url_enedis):
    
    enedis_with_ban_data = get_enedis_with_ban_pandas(requete_url_enedis)
    
    ademe_data = []
    ademe_data_res = [requests.get(get_url_ademe_filter_on_ban(_)).json().get('results') for _ in enedis_with_ban_data.id_BAN.values.tolist()] # liste à 2 niveaux # pour chaque Id_BAN on a plusieurs lignes ademe
    for _ in ademe_data_res:
        ademe_data.extend(_)
    ademe_data = pd.DataFrame(ademe_data)

    ademe_data = ademe_data.add_suffix('_ademe')
    enedis_with_ban_data = enedis_with_ban_data.add_suffix('_enedis_with_ban')

    del ademe_data_res
    return pd.merge(ademe_data,
                    enedis_with_ban_data,
                    how='left',
                    left_on='Identifiant__BAN_ademe',
                    right_on='id_BAN_enedis_with_ban')

tmp = get_enedis_with_ban_with_ademe(get_url_enedis_year_rows(2018, 10))

In [133]:
[_ for _ in tmp.columns if 'dpe' in _.lower()]

['Date_établissement_DPE_ademe',
 'N°DPE_ademe',
 'Modèle_DPE_ademe',
 'Méthode_application_DPE_ademe',
 'Date_réception_DPE_ademe',
 'Date_fin_validité_DPE_ademe',
 'Version_DPE_ademe',
 'Etiquette_DPE_ademe',
 'N°_DPE_immeuble_associé_ademe',
 'N°_DPE_remplacé_ademe']

In [134]:
# tmp.Etiquette_DPE.value_counts(dropna=False)
sorted(list(tmp.columns))

['Adresse_(BAN)_ademe',
 'Adresse_brute_ademe',
 'Année_construction_ademe',
 'Appartement_non_visité_(0/1)_ademe',
 'Apports_internes_saison_chauffe__ademe',
 'Apports_internes_saison_froid_ademe',
 'Apports_solaires_saison_chauffe_ademe',
 'Apports_solaires_saison_froid_ademe',
 'Besoin_ECS_ademe',
 'Besoin_chauffage_ademe',
 'Besoin_refroidissement_ademe',
 'Besoin_refroidissement_dépensier_ademe',
 'Catégorie_ENR_ademe',
 'Classe_altitude_ademe',
 'Classe_inertie_bâtiment_ademe',
 'Code_INSEE_(BAN)_ademe',
 'Code_postal_(BAN)_ademe',
 'Code_postal_(brut)_ademe',
 "Complément_d'adresse_bâtiment_ademe",
 "Complément_d'adresse_logement_ademe",
 'Configuration_installation_ECS_ademe',
 'Configuration_installation_chauffage_n°1_ademe',
 'Conso_5_usages/m²_é_finale_ademe',
 'Conso_5_usages_par_m²_é_primaire_ademe',
 'Conso_5_usages_é_finale_ademe',
 'Conso_5_usages_é_finale_énergie_n°1_ademe',
 'Conso_5_usages_é_finale_énergie_n°2_ademe',
 'Conso_5_usages_é_primaire_ademe',
 'Conso_ECS_d

In [ ]:
class DataEnedisAdeme:
    def __init__(self):
        pass

    

    def get_enedis_with_ban_with_ademe(requete_url_enedis):
        
        enedis_with_ban_data = get_enedis_with_ban_pandas(requete_url_enedis)
        
        ademe_data = []
        ademe_data_res = [requests.get(get_url_ademe_filter_on_ban(_)).json().get('results') for _ in enedis_with_ban_data.id_BAN.values.tolist()] # liste à 2 niveaux # pour chaque Id_BAN on a plusieurs lignes ademe
        for _ in ademe_data_res:
            ademe_data.extend(_)
        ademe_data = pd.DataFrame(ademe_data)

        ademe_data = ademe_data.add_suffix('_ademe')
        enedis_with_ban_data = enedis_with_ban_data.add_suffix('_enedis_with_ban')

        del ademe_data_res
        return pd.merge(ademe_data,
                        enedis_with_ban_data,
                        how='left',
                        left_on='Identifiant__BAN_ademe',
                        right_on='id_BAN_enedis_with_ban')

    def extract_year_rows(self, year:int=2018, rows:int=20):
        pass